In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

## Basic app to interact with API

In [3]:
# from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

In [4]:
# llm = OpenAI()
llm = ChatOpenAI(model=MODEL_GPT)

### Define API documentation
- We are going to use short version of RestCountries API. [REST Countries](https://restcountries.com/) 
- Important note: our app will only work with endpoints we define below, althougt original API has many more endpoints.

In [5]:
api_docs = """
BASE URL: https://restcountries.com/

API Documentation:

The API endpoint /v3.1/name/{name} Used to find informatin about 
a country. All URL parameters are listed below:
    - name: Name of country - Example: Italy, France
    
The API endpoint /v3.1/currency/{currency} Used to find information 
about a region. All URL parameters are listed below:
    - currency: 3 letter currency. Example: USD, COP

The API endpoint /v3.1/lang/{language} Used to find information 
about the official language of the country. All URL parameters 
are listed below:
    - language: language of the country. Example: English, Spanish
    
"""

### Create chain to read API documentation

In [6]:
from langchain.chains import APIChain

In [7]:
api_chain = APIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=api_docs,
    verbose=True,
    limit_to_domains=["https://restcountries.com/"]
)

### Ask question about API

In [8]:
question = "Give me information about France in less than 100 words."

In [9]:
# api_chain.run(question)
api_chain.invoke(question)



> Entering new APIChain chain...
To get information about France using the provided API documentation, the appropriate API endpoint would be:

API url: `https://restcountries.com/v3.1/name/France`


ValueError: To get information about France using the provided API documentation, the appropriate API endpoint would be:

API url: `https://restcountries.com/v3.1/name/France` is not in the allowed domains: ['https://restcountries.com/']

In [10]:
question2 = """
List the top 3 biggest countries 
where the official language is French.
"""

In [11]:
api_chain.run(question2)

C:\Users\Pavel\AppData\Local\Temp\ipykernel_11572\1031627936.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  api_chain.run(question2)




> Entering new APIChain chain...
To answer the question about the top 3 biggest countries where the official language is French, we can break it down into a few steps. 

1. We need to find countries where French is the official language.
2. We can then filter those countries based on their size (land area).

Given the API endpoints provided, we will first query the countries by language:

### Step 1: API Call for French Language
We will use the `/v3.1/lang/{language}` endpoint to find the countries that have French as an official language.

The API URL will be:
```
https://restcountries.com/v3.1/lang/French
```

This URL will return information about all countries where French is spoken, and we can then determine the top 3 biggest countries from that response based on the size data provided.

### Final API URL:
```
https://restcountries.com/v3.1/lang/French
``` 

This is the API URL you would use to gather the necessary information to answer the user's question.


ValueError: To answer the question about the top 3 biggest countries where the official language is French, we can break it down into a few steps. 

1. We need to find countries where French is the official language.
2. We can then filter those countries based on their size (land area).

Given the API endpoints provided, we will first query the countries by language:

### Step 1: API Call for French Language
We will use the `/v3.1/lang/{language}` endpoint to find the countries that have French as an official language.

The API URL will be:
```
https://restcountries.com/v3.1/lang/French
```

This URL will return information about all countries where French is spoken, and we can then determine the top 3 biggest countries from that response based on the size data provided.

### Final API URL:
```
https://restcountries.com/v3.1/lang/French
``` 

This is the API URL you would use to gather the necessary information to answer the user's question. is not in the allowed domains: ['https://res

## APIChain
https://python.langchain.com/api_reference/langchain/chains/langchain.chains.api.base.APIChain.html

In [12]:
# !pip install -U langgraph

In [13]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain.chains.api.prompt import API_URL_PROMPT
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.utilities.requests import TextRequestsWrapper
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode

# NOTE: There are inherent risks in giving models discretion
# to execute real-world actions. We must "opt-in" to these
# risks by setting allow_dangerous_request=True to use these tools.
# This can be dangerous for calling unwanted requests. Please make
# sure your custom OpenAPI spec (yaml) is safe and that permissions
# associated with the tools are narrowly-scoped.
ALLOW_DANGEROUS_REQUESTS = True

# Subset of spec for https://jsonplaceholder.typicode.com
api_spec = """
openapi: 3.0.0
info:
  title: JSONPlaceholder API
  version: 1.0.0
servers:
  - url: https://jsonplaceholder.typicode.com
paths:
  /posts:
    get:
      summary: Get posts
      parameters: &id001
        - name: _limit
          in: query
          required: false
          schema:
            type: integer
          example: 2
          description: Limit the number of results
"""

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
toolkit = RequestsToolkit(
    requests_wrapper=TextRequestsWrapper(headers={}),  # no auth required
    allow_dangerous_requests=ALLOW_DANGEROUS_REQUESTS,
)
tools = toolkit.get_tools()

api_request_chain = (
    API_URL_PROMPT.partial(api_docs=api_spec)
    | llm.bind_tools(tools, tool_choice="any")
)

class ChainState(TypedDict):
    """LangGraph state."""

    messages: Annotated[Sequence[BaseMessage], add_messages]


async def acall_request_chain(state: ChainState, config: RunnableConfig):
    last_message = state["messages"][-1]
    response = await api_request_chain.ainvoke(
        {"question": last_message.content}, config
    )
    return {"messages": [response]}

async def acall_model(state: ChainState, config: RunnableConfig):
    response = await llm.ainvoke(state["messages"], config)
    return {"messages": [response]}

graph_builder = StateGraph(ChainState)
graph_builder.add_node("call_tool", acall_request_chain)
graph_builder.add_node("execute_tool", ToolNode(tools))
graph_builder.add_node("call_model", acall_model)
graph_builder.set_entry_point("call_tool")
graph_builder.add_edge("call_tool", "execute_tool")
graph_builder.add_edge("execute_tool", "call_model")
graph_builder.add_edge("call_model", END)
chain = graph_builder.compile()

In [14]:
example_query = "Fetch the top two posts. What are their titles?"

events = chain.astream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
async for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Fetch the top two posts. What are their titles?
================================== Ai Message ==================================
Tool Calls:
  requests_get (call_iCh6lVO28b7BmVSdsqiQ6Wvy)
 Call ID: call_iCh6lVO28b7BmVSdsqiQ6Wvy
  Args:
    url: https://jsonplaceholder.typicode.com/posts?_limit=2
================================= Tool Message =================================
Name: requests_get

[
  {
    "userId": 1,
    "id": 1,
    "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
    "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
  },
  {
    "userId": 1,
    "id": 2,
    "title": "qui est esse",
    "body": "est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil moles